In [9]:
# %%
import sqlite3

# Connect to the SQLite database
db_path = r'F:\ML\PythonAIProject\SMARKMediaTools_web\electron-media-toolbox\.cache\photos.db'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Print the list of tables in the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Tables in the database:", tables)

# %%
import pandas as pd

# Query all records from the 'present' table
cursor.execute("SELECT * FROM present")
present_data = cursor.fetchall()

# Get column names
cursor.execute("PRAGMA table_info(present)")
columns = [column[1] for column in cursor.fetchall()]

# Create a pandas DataFrame for better visualization
df_present = pd.DataFrame(present_data, columns=columns)
print("\nFirst few rows of 'present' table:")
print(df_present.head())

# %%
# Sort DataFrame by filePath
df_present_sorted = df_present.sort_values('filePath')

# %%
df_present_sorted

Tables in the database: [('present',), ('sqlite_sequence',), ('previous',)]

First few rows of 'present' table:
   id                fileName  \
0   1  QQ图片20230605172326.jpg   
1   2                 晡时6.jpg   
2   3          DSC02352co.jpg   
3   4          DSC02354cq.jpg   
4   5          DSC02340cg.jpg   

                                             fileUrl  \
0  thumbnail-resource://G:/datasets/dataset/QQ图片2...   
1   thumbnail-resource://G:/datasets/dataset/晡时6.jpg   
2  thumbnail-resource://G:/datasets/dataset/DSC02...   
3  thumbnail-resource://G:/datasets/dataset/DSC02...   
4  thumbnail-resource://G:/datasets/dataset/DSC02...   

                                     filePath    fileSize  \
0  G:/datasets/dataset/QQ图片20230605172326.jpg    894054.0   
1                 G:/datasets/dataset/晡时6.jpg  77540215.0   
2          G:/datasets/dataset/DSC02352co.jpg  48544159.0   
3          G:/datasets/dataset/DSC02354cq.jpg  47535460.0   
4          G:/datasets/dataset/DSC02340cg.jpg  

,id,fileName,fileUrl,filePath,fileSize,info,date,groupId,simRefPath,similarity,IQA,isEnabled
2269,2270,-459854a6c19e0454.jpg,thumbnail-resource://G:/datasets/dataset/-4598...,G:/datasets/dataset/-459854a6c19e0454.jpg,NaN,None,None,2075,G:/datasets/dataset/Z30_4660.jpg,0.307425,71.424069,1
752,753,01 绽放.jpg,thumbnail-resource://G:/datasets/dataset/01 绽放...,G:/datasets/dataset/01 绽放.jpg,12037006.0,1/NaN undefined,Invalid Date,646,G:/datasets/dataset/8P6A9768.JPG,0.026043,74.025960,1
1849,1850,02 日出.jpg,thumbnail-resource://G:/datasets/dataset/02 日出...,G:/datasets/dataset/02 日出.jpg,NaN,None,None,1658,G:/datasets/dataset/Z30_2068-NEF_DxO_DeepPRIME...,0.180869,53.331599,1
1730,1731,03 生机.jpg,thumbnail-resource://G:/datasets/dataset/03 生机...,G:/datasets/dataset/03 生机.jpg,NaN,None,None,1543,G:/datasets/dataset/Z30_3484.JPG,0.047309,58.020205,1
2143,2144,04 清晨.jpg,thumbnail-resource://G:/datasets/dataset/04 清晨...,G:/datasets/dataset/04 清晨.jpg,NaN,None,None,1951,G:/datasets/dataset/Z30_4720.jpg,0.072482,58.185925,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1617,1618,跳远6.jpg,thumbnail-resource://G:/datasets/dataset/跳远6.jpg,G:/datasets/dataset/跳远6.jpg,NaN,None,None,1440,G:/datasets/dataset/8P6A9940.JPG,0.350824,67.767129,1
1640,1641,选手合影1.JPG,thumbnail-resource://G:/datasets/dataset/选手合影1...,G:/datasets/dataset/选手合影1.JPG,NaN,None,None,1460,G:/datasets/dataset/20190320_173155_mix01.jpg,-0.043713,63.492069,1
249,250,鸭子.JPG,thumbnail-resource://G:/datasets/dataset/鸭子.JPG,G:/datasets/dataset/鸭子.JPG,20955515.0,1/200 FE 70-200mm F2.8 GM OSS II,2023/6/18 02:46:47,214,G:/datasets/dataset/猫 (6).JPG,0.685662,68.901601,1
459,460,鹅.jpg,thumbnail-resource://G:/datasets/dataset/鹅.jpg,G:/datasets/dataset/鹅.jpg,17814042.0,1/250 FE 50mm F1.2 GM,2023/5/4 02:30:36,394,G:/datasets/dataset/DSC00383.JPG,0.223999,64.390907,1


In [12]:
import sqlite3
import pandas as pd


def robust_sort_and_update(db_path):
    # 创建数据库连接
    conn = sqlite3.connect(db_path)

    try:
        # 第一步：读取整个表到DataFrame
        df = pd.read_sql("SELECT * FROM present", conn)

        # 第二步：按filePath排序并重置ID
        df_sorted = df.sort_values('filePath',ascending=True).reset_index(drop=True)
        df_sorted['id'] = df_sorted.index + 1  # 生成连续新ID

        # 第三步：事务处理更新
        with conn:
            cursor = conn.cursor()

            # 清空原表（保留表结构）
            cursor.execute("DELETE FROM present")

            # 动态生成插入语句
            columns = ', '.join(df.columns)
            placeholders = ', '.join(['?'] * len(df.columns))
            insert_sql = f"INSERT INTO present ({columns}) VALUES ({placeholders})"

            # 批量插入排序后的数据
            cursor.executemany(insert_sql, df_sorted.itertuples(index=False, name=None))

        # 第四步：验证更新
        verify_df = pd.read_sql("SELECT * FROM present ORDER BY id", conn)
        print("更新验证：")
        print(f"总记录数：{len(verify_df)}")
        print("前5条记录：")
        print(verify_df[['id', 'filePath']].head())

    finally:
        conn.close()


# 使用示例
db_path = r'F:\ML\PythonAIProject\SMARKMediaTools_web\electron-media-toolbox\.cache\photos.db'
robust_sort_and_update(db_path)

更新验证：
总记录数：2590
前5条记录：
   id                                   filePath
0   1  G:/datasets/dataset/-459854a6c19e0454.jpg
1   2              G:/datasets/dataset/01 绽放.jpg
2   3              G:/datasets/dataset/02 日出.jpg
3   4              G:/datasets/dataset/03 生机.jpg
4   5              G:/datasets/dataset/04 清晨.jpg
